# Úkol č. 1 - vizualizace dat a web scraping (do 11. října)

  * V rámci tohoto úkolu musíte stáhnout dat z webu (tzv. _web scraping_, velmi základní) a následně data zpracovat a vizualizovat.
  * Cílem bude stáhnout data ze serveru https://www.psp.cz/sqw/hlasovani.sqw?o=8 týkající hlasování současné poslanecké sněmovny, uložit tato data v tabulkovém formátu a pak vymyslet vizualizace a zobrazení dat, které umožní orientaci v těchto datech a zvýrazní zajímavé informace a zobrazit přehledně časový vývoj různých veličin.
 
> **Úkoly jsou zadány tak, aby Vám daly prostor pro invenci. Vymyslet _jak přesně_ budete úkol řešit, je důležitou součástí zadání a originalita či nápaditost bude také hodnocena!**

## Data

 * Měli byste stáhnout data ze všech hlasování současné poslanecké sněmovny (tj. od voleb v roce 2017) a to až na úroveň jednotlivých poslanců. 
 * Data by měla obsahovat i stručný popis toho, o jaké hlasování šlo.

## Pokyny k vypracování

**Základní body zadání**, za jejichž (poctivé) vypracování získáte **8 bodů**:
  * Strojově stáhněte data a uložte je do vhodného formátu, který se Vám bude dále dobře zpracovávat. 
  * **I když nemusíte mít sněmovnu rádi, snažte se nezahltit server a mezi požadavky na stránku mějte alespoň vteřinu mezeru.**
  * V druhé části Vašeho Jupyter notebooku pracujte se staženými daty v souborech, které jsou výsledkem Vašeho stahovacího skriptu (aby opravující nemusel spouštět stahování z webu).
  * S využitím vybraných nástrojů zpracujte data a vymyslete vizualizace a grafy, aby bylo vidět následující:
    * Odchody a přeběhnutí poslanců mezi jednotlivými stranami.
    * Jakou mají jednotliví poslanci docházku (účast na hlasování) a jak jsou na tom s docházkou strany jako celek.
    * Jak často spolu jednotlivé strany hlasují ve shodě a jak často hlasují odlišně.
    * Jak jsou poslanci jednotlivých stran jednotní v hlasování (jak často hlasují stejně, kdo jsou největší rebelové).

**Další body zadání** za případné další body (můžete si vybrat, maximum bodů za úkol je každopádně 12 bodů):
  * Ve vizualizacích a grafech nějak zachyťte časový vývoj (např. jak se mění docházka, shoda mezi stranami atp.).
  * Najděte jednotlivé poslance, kteří se nejvíce shodují při svém hlasování či kteří mají co nejpodobnější účast na hlasování.
  * Zkuste rozlišit v datech důležitá hlasování a zpracujte vizualizace (také) pro ně.
  * Zkuste najít hlasování, kde poslanci hlasovali nejvíce jinak, než obvykle.
  
## Tipy a triky
  * Balíčky iportujte na začátku notebooku (příp. na začátku scrapovací a pak na začátku vizualizační části).
  * Využívejte markdown buňky (jako je ta, ve které je tento text) a naspisy, abyste usnadnily orientaci opravující(mu).
  * Pečlivě si vybírejte grafy a vizualizace, aby co nejlépe vyjádřily to, co vyjádřit chcete. Doporučuji projít si pro inspiraci alespoň galerie balíčků `matplotlib` a `seaborn`.

## Poznámky k odevzdání

  * Řiďte se pokyny ze stránky https://courses.fit.cvut.cz/BI-VZD/homeworks/index.html.
  * Odevzdejte Jupyter Notebook (příp. doplněn skripty), ale i soubor(y) se staženými daty (ať opravující nemusí stahovat data).
  * Opravující Vám může umožnit úkol dodělat či opravit a získat tak další body. První verze je ale důležitá a bude-li odbytá, budete za to penalizováni.

# Solution
## Data extraction
First - let's get which schuze are available

In [268]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
url = " https://www.psp.cz/sqw/hlasovani.sqw?o=8"
page = requests.get(url)
soap = BeautifulSoup(page.text, "html.parser")
schuze_pre=soap.select("tbody > tr td.col-number a")
schuze=[]
for tag in schuze_pre:
    schuze.append(tag.text.split('.')[0])
print('all available schuze:\n',schuze)

all available schuze:
 ['1', '2', '3', '4', '5', '6', '7', '8', '9', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61']


### Prepare for extracting data to hlasovani and schuze dataframes
dataframes[0] is a schuze dataframe, which'll be written to schuze.csv

dataframes[1] is a hlasovani dataframe - hlasovani.csv

following method retrieves data from hlasovani page (example: https://www.psp.cz/sqw/hlasy.sqw?g=72722&l=cz)

In [269]:
import time
def process_hlasovani(html_hlasovani, dataframes, nschuze, nhlasovani):
    soap = BeautifulSoup(html_hlasovani, 'html.parser')
    tmp = soap.select("div#main-content > h2 > span")[:-1]
    nazvy_klubu = []
    for nazev in tmp:
        nazvy_klubu.append(nazev.text.split(' ')[0])
        
    cleny_klubu = soap.select("div#main-content > ul.results")
    for i in range(len(nazvy_klubu)):
        for person in cleny_klubu[i].select('li'):
            dataframes[1] = dataframes[1].append({
                'Schůze': nschuze,
                'Č. hl.': nhlasovani,
                'Klub': nazvy_klubu[i],
                'Jmeno': person.a.text,
                'Rozhodnuti': person.span.text
            }, ignore_index = True)
    time.sleep(0.5)

next function retrieves the table from this kind of pages: https://www.psp.cz/sqw/phlasa.sqw?o=8&s=44&l=cz&pg=9
and extracts all hlasovani, linked from this page, using the **`process_hlasovani()`** above

In [270]:
hlasovani_url_base = 'https://www.psp.cz/sqw/'
def process_schuze_page(html_schuze_page, dataframes, nschuze):
    # update schuze dataframe (dataframes[0])
    local_df = pd.read_html(html_schuze_page)[0]
    dataframes[0] = dataframes[0].append(local_df, ignore_index = True)
    
    # update hlasovani dataframe (dataframes[1])
    soap = BeautifulSoup(html_schuze_page,'html.parser')
    jednotliva_hlasovani = soap.select('div.search-results tr > td > a[href^="hlasy"]')
    # for each voting - update dataframe
    for hlasovani in jednotliva_hlasovani:
        process_hlasovani(
            requests.get(hlasovani_url_base + hlasovani['href']).text,
            dataframes,
            nschuze,
            hlasovani.text
        )

### Link what is done
We can process hlasovani page and one schuze page of hlasovani. In order to round everything up we need:
* determine amount of pages for currently processed schuze
* visit each schuze page and apply `process_schuze_page()` to it
* repeat steps above for each schuze from `schuze` list, collected at the beginning of this file

In [287]:
# determine amount of pages of hlasovani table for POST request
def amount_of_pages(html_first_schuze_page):
    soap = BeautifulSoup(html_first_schuze_page,'html.parser')
    selection = soap.select("div#main-content > div.search-results > p.paging > span.pages > a")
    if not selection:
        return 1
    elif selection[-1].text == '>>':
        return int(selection[-1]['href'].split('=')[-1])
    else:
        return int(selection[-2].text)




schuze_post_url='https://www.psp.cz/sqw/phlasa.sqw'

def process_schuze(schuze_number, file_write_mode, write_header):
    # clear container dataframes for each schuze, or pretty soon append() operation will take too much time
    container =[
        pd.DataFrame(columns = ['Schůze','Č. hl.','Bod','Název','Datum','Výsledek']),
        pd.DataFrame(columns = ['Schůze','Č. hl.','Klub','Jmeno','Rozhodnuti'])
    ]
    
    # post request data
    schuze_post_data={
        'o': '8',
        's': schuze_number,
        'pg': '1'
    }
    
    print('\n\nschuze',schuze_number,'started processing')
    r = requests.post(schuze_post_url, schuze_post_data)
    npages = amount_of_pages(r.text)
    print('schuzka',schuze_number,': amount of pages:', npages)
    
    # process first page of hlasovani table (https://www.psp.cz/sqw/phlasa.sqw?o=8&s=44&l=cz&pg=9)
    # first page is processed separately because it's already downloaded
    process_schuze_page(r.text, container, schuze_number)
    # process other pages of hlasovani table
    for page in range(2, npages + 1):
        schuze_post_data['pg'] = str(page)
        r = requests.post(schuze_post_url, schuze_post_data)
        process_schuze_page(r.text, container, schuze_number)
        
    print('schuze',schuze_number,'processed...saving')
    container[0].to_csv('schuze.csv',index=False, mode=file_write_mode, header=write_header)
    container[1].to_csv('hlasovani.csv',index=False, mode=file_write_mode, header=write_header)
    print(schuze_number,'saved')

### Fire

In [288]:
# first schuze is processed separately becauze I want dataframe headers to appear only once (first line of the file),
# and it is desired to make new .csv file each time the procedure is launched
process_schuze(schuze[0],'w',True)
for i in range(1,len(schuze)):
    process_schuze(schuze[i],'a',False)



schuze 1 started processing
schuzka 1 : amount of pages: 1
schuze 1 processed...saving
1 saved


schuze 2 started processing
schuzka 2 : amount of pages: 4
schuze 2 processed...saving
2 saved


schuze 3 started processing
schuzka 3 : amount of pages: 1
schuze 3 processed...saving
3 saved


schuze 4 started processing
schuzka 4 : amount of pages: 7
schuze 4 processed...saving
4 saved


schuze 5 started processing
schuzka 5 : amount of pages: 1
schuze 5 processed...saving
5 saved


schuze 6 started processing
schuzka 6 : amount of pages: 5
schuze 6 processed...saving
6 saved


schuze 7 started processing
schuzka 7 : amount of pages: 8
schuze 7 processed...saving
7 saved


schuze 8 started processing
schuzka 8 : amount of pages: 1
schuze 8 processed...saving
8 saved


schuze 9 started processing
schuzka 9 : amount of pages: 1
schuze 9 processed...saving
9 saved


schuze 11 started processing
schuzka 11 : amount of pages: 1
schuze 11 processed...saving
11 saved


schuze 12 started proces

## Working with data
By now my custom crawler have extracted all the info needed for exploration (it took around 4-5 hours).
Let's put collected data from files into dataframes to easily work with them

In [290]:
hlasy = pd.read_csv('hlasovani.csv')
display(hlasy)

,Schůze,Č. hl.,Klub,Jmeno,Rozhodnuti
0,1,1,ANO,Věra Adámková,A
1,1,1,ANO,Andrej Babiš,A
2,1,1,ANO,Andrea Babišová,A
3,1,1,ANO,Margita Balaštíková,A
4,1,1,ANO,Josef Bělica,A
...,...,...,...,...,...
1395995,61,46,Nezařaz,Tereza Hyťhová,M
1395996,61,46,Nezařaz,Václav Klaus,M
1395997,61,46,Nezařaz,Zuzana Majerová Zahradníková,M
1395998,61,46,Nezařaz,Ivana Nevludová,M


In [291]:
schuze = pd.read_csv('schuze.csv')
display(schuze)

,Schůze,Č. hl.,Bod,Název,Datum,Výsledek
0,1,1,3.0,Inf. o ustavení volební komise PS a volbě členů,20. 11. 2017 13:53,Přijato (zmatečné)
1,1,2,3.0,Inf. o ustavení volební komise PS a volbě členů,20. 11. 2017 13:53,Přijato
2,1,3,NaN,Procedurální hlasování,20. 11. 2017 14:47,Zamítnuto
3,1,4,4.0,Náv.na stanovení počtu čl. MIV,20. 11. 2017 15:39,Zamítnuto
4,1,5,4.0,Náv.na stanovení počtu čl. MIV,20. 11. 2017 15:39,Přijato
...,...,...,...,...,...,...
6978,61,42,6.0,Novela z. o Vojenském zpravodajství (sněmovní ...,7. 10. 2020 17:54,Přijato
6979,61,43,7.0,Novela z. - občanský soudní řád (sněmovní tisk...,7. 10. 2020 18:43,Přijato
6980,61,44,7.0,Novela z. - občanský soudní řád (sněmovní tisk...,7. 10. 2020 18:44,Přijato
6981,61,45,8.0,Novela z. o platu představitelů státní moci (s...,7. 10. 2020 19:23,Přijato
